In [5]:
import requests
with open("secret.csv", "r") as r:
    data = [line.replace("\n","").split(",") for line in r.readlines()]
    username = data[0][1]
    token = data[1][1]

In [7]:
token

'YmjqaQBwC9nvdt0C'

In [8]:
url = "https://lichess.org"
resp = requests.get(
  f'https://lichess.org/api/games/user/{username}',
  params={
    'rated': False,
    "perfType" : "blitz",
    "moves" : False,
    "opening" : True
  },
  headers={
    'Authorization': f'Bearer {token}' # Need this or you will get a 401: Not Authorized response
  }
)